<a href="https://colab.research.google.com/github/PiyumaliSandunika/e18-4yp-Multimodal-Emotion-Prediction-Using-Reinforcement-Learning/blob/main/model_ensemble_weighted_avg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# label_map = {"happiness": 0, "sadness": 1, "anger": 2, "neutral": 3}


** Model Loading**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import torch
import pickle
# Load the saved model
model_path = "/content/drive/MyDrive/FYP_Text/emotion_text_classification_model.pkl"
# loaded_model = DistilBertForSequenceClassification.from_pretrained(model_path)
with open(model_path,'rb') as f:
  loaded_model = pickle.load(f)

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


In [ ]:
model_path2 = "emotion_classification_model_lite"
loaded_model2 = DistilBertForSequenceClassification.from_pretrained(model_path)
# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


In [ ]:
def predict_emotion_with_probabilities(sentence, model, tokenizer):
    # Tokenize the input sentence
    inputs = tokenizer(sentence, return_tensors="pt")

    # Perform inference
    outputs = model(**inputs)
    logits = outputs.logits

    # Apply softmax to get probabilities
    probabilities = torch.nn.functional.softmax(logits, dim=1)

    # Convert probabilities tensor to list
    probabilities_list = probabilities.squeeze().tolist()

    # Map predicted label to emotion
    emotion_labels = ["happiness", "sadness", "anger", "neutral"]
    predicted_emotion = emotion_labels[torch.argmax(logits, dim=1).item()]

    return probabilities_list, predicted_emotion


In [ ]:
def ensemble_predictions(sentence, model_weights=None, class_weights=None):
    # Get predictions from the first model
    probabilities1, _ = predict_emotion_with_probabilities(sentence, loaded_model, tokenizer)

    # Get predictions from the second model
    probabilities2, _ = predict_emotion_with_probabilities(sentence, loaded_model2, tokenizer)

    # Apply model-wise weights if provided
    if model_weights:
        probabilities1 = [p1 * w for p1, w in zip(probabilities1, model_weights[0])]
        probabilities2 = [p2 * w for p2, w in zip(probabilities2, model_weights[1])]

    # Apply class-wise weights if provided
    if class_weights:
        probabilities1 = [p1 * w for p1, w in zip(probabilities1, class_weights)]
        probabilities2 = [p2 * w for p2, w in zip(probabilities2, class_weights)]

    # Calculate ensemble probabilities by averaging
    ensemble_probabilities = [(p1 + p2) / 2 for p1, p2 in zip(probabilities1, probabilities2)]

    # Map ensemble probabilities to emotion labels
    emotion_labels = ["happiness", "sadness", "anger", "neutral"]
    predicted_emotion = emotion_labels[np.argmax(ensemble_probabilities)]

    return probabilities1, probabilities2, ensemble_probabilities, predicted_emotion

# Example usage with model and class weights
sentence = "This is a test sentence."
model_weights = [[1.0, 1.0, 1.0, 1.0], [0.8, 0.8, 0.9, 1.2]]  # Example weights for each model
class_weights = [1.0, 1.0, 0.8, 1.2]  # Example weights for each class
probabilities1, probabilities2, ensemble_probabilities, predicted_emotion = ensemble_predictions(sentence, model_weights, class_weights)
print("Probabilities (Model 1):", probabilities1)
print("Probabilities (Model 2):", probabilities2)
print("Ensemble Probabilities:", ensemble_probabilities)
print("Ensemble Predicted Emotion:", predicted_emotion)


Probabilities (Model 1): [0.0003059331502299756, 0.0008683084743097425, 0.0019081057980656624, 1.1957287073135376]
Probabilities (Model 2): [0.0002447465201839805, 0.000694646779447794, 0.0017172952182590963, 1.434874448776245]
Ensemble Probabilities: [0.000275339835206978, 0.0007814776268787682, 0.0018127005081623794, 1.3153015780448913]
Ensemble Predicted Emotion: neutral
